In [25]:
import pandas as pd
import numpy as np
import plotly.express as px


# Carregar os dados das tabelas
df_orders = pd.read_csv('assets/olist_orders_dataset.csv', encoding='latin1')
df_items = pd.read_csv('assets/olist_order_items_dataset.csv', encoding='latin1')
df_geolocation = pd.read_csv('assets/olist_geolocation_dataset.csv', encoding='latin1')
df_reviews = pd.read_csv('assets/olist_order_reviews_dataset.csv', encoding='latin1')
df_customers = pd.read_csv('assets/olist_customers_dataset.csv', encoding='latin1')


df_items = df_items.dropna()
df_geolocation = df_geolocation.dropna()
df_customers = df_customers.dropna()

# Renomear colunas olist_orders_dataset
df_orders = df_orders.rename(columns={
    'order_id': 'ID_pedido',
    'customer_id': 'ID_cliente',
    'order_status': 'Status_pedido',
    'order_purchase_timestamp': 'Data_da_compra',
    'order_approved_at': 'Pedido_aprovado_em',
    'order_delivered_carrier_date': 'Pedido_entregue_data_transportadora',
    'order_delivered_customer_date': 'Pedido_entregue_ao_cliente',
    'order_estimated_delivery_date': 'Data_estimada_de_entrega'
})

# Renomear colunas olist_orders_dataset
df_items = df_items.rename(columns={
    'order_id': 'ID_pedido',
    'order_item_id': 'ID_item_pedido',
    'product_id': 'ID_produto',
    'seller_id': 'ID_vendedor',
    'shipping_limit_date': 'Data_Limite_de_envio',
    'price': 'Preço',
    'freight_value': 'Frete'
})
df_items = df_items.dropna()

# Renomear colunas olist_geolocation_dataset
df_geolocation = df_geolocation.rename(columns={
    'geolocation_zip_code_prefix': 'Prefixo_CEP',
    'geolocation_lng': 'Longitude',
    'geolocation_lat': 'Latitude',
    'geolocation_city': 'Cidade',
    'geolocation_state': 'Estado'
})
df_geolocation = df_geolocation.dropna()

# Renomear colunas olist_order_reviews_dataset
df_reviews = df_reviews.rename(columns={
    'review_id': 'ID_de_revisao',
    'order_id': 'ID_pedido',
    'review_score': 'Pontuacao_de_revisao',
    'review_comment_title': 'Titulo_de_comentario',
    'review_comment_message': 'Comentario',
    'review_creation_date': 'Data_de_criacao',
    'review_answer_timestamp': 'Data_de_registro'
})
df_reviews = df_reviews.dropna()

# Renomear colunas olist_customers_dataset
df_customers = df_customers.rename(columns={
    'customer_id': 'ID_cliente',
    'customer_unique_id': 'ID_cliente_unico',
    'customer_zip_code_prefix': 'CEP_cliente',
    'customer_city': 'Cidade_cliente',
    'customer_state': 'Estado_cliente'
})

# Unir as tabelas df_orders, df_items, df_reviews e df_customers
df_combined = df_orders.merge(df_items, on='ID_pedido')
df_combined = df_combined.merge(df_reviews, on='ID_pedido')
df_combined = df_combined.merge(df_customers, on='ID_cliente')

# Unir com a tabela de geolocalização (usaremos apenas uma amostra da geolocalização para simplificação)
df_geolocation_sample = df_geolocation.drop_duplicates(subset=['Prefixo_CEP'])
df_combined = df_combined.merge(df_geolocation_sample, left_on='CEP_cliente', right_on='Prefixo_CEP')

# Função para calcular a distância de Haversine
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Raio da Terra em quilômetros
    return c * r

# Supondo que temos a latitude e longitude do centro de distribuição (ou armazém)
warehouse_lat = -22.550520 
warehouse_lon = -36.633308  

# Calcular a distância de entrega para cada pedido
df_combined['Distancia_entrega'] = df_combined.apply(lambda row: haversine(warehouse_lon, warehouse_lat, row['Longitude'], row['Latitude']), axis=1)


display(df_combined.head())

# Criar o gráfico de dispersão com Plotly Express
graph = px.scatter(df_combined, x='Distancia_entrega', y='Frete', color='Pontuacao_de_revisao',
                 title='Relação entre Custos de Frete e Distância de Entrega',
                 labels={'Distancia_entrega': 'Distância de Entrega (km)', 'Frete': 'Frete (R$)'},
                 color_continuous_scale='viridis')

# Adicionar título aos eixos
graph.update_xaxes(title_text='Distância de Entrega (km)')
graph.update_yaxes(title_text='Frete (R$)')

graph.show()

,ID_pedido,ID_cliente,Status_pedido,Data_da_compra,Pedido_aprovado_em,Pedido_entregue_data_transportadora,Pedido_entregue_ao_cliente,Data_estimada_de_entrega,ID_item_pedido,ID_produto,...,ID_cliente_unico,CEP_cliente,Cidade_cliente,Estado_cliente,Prefixo_CEP,Latitude,Longitude,Cidade,Estado,Distancia_entrega
0,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,...,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,47813,-12.169860,-44.988369,barreiras,BA,1454.550963
1,dcb36b511fcac050b97cd5c05de84dc3,3b6828a50ffe546942b7a473d70ac0fc,delivered,2018-06-07 19:03:12,2018-06-12 23:31:02,2018-06-11 14:54:00,2018-06-21 15:34:32,2018-07-04 00:00:00,1,009c09f439988bc06a93d6b8186dce73,...,ccafc1c3f270410521c3c6f3b249870f,74820,goiania,GO,74820,-16.712157,-49.248251,goiania,GO,1471.085157
2,f3e7c359154d965827355f39d6b1fdac,62b423aab58096ca514ba6aa06be2f98,delivered,2018-08-09 11:44:40,2018-08-10 03:24:51,2018-08-10 12:29:00,2018-08-13 18:24:27,2018-08-17 00:00:00,1,e99d69efe684efaa643f99805f7c81bc,...,9c9242ad7f1b52d926ea76778e1c0c57,18052,sorocaba,SP,18052,-23.539013,-47.491042,sorocaba,SP,1116.128183
3,acce194856392f074dbf9dada14d8d82,7e20bf5ca92da68200643bda76c504c6,delivered,2018-06-04 00:00:13,2018-06-05 00:35:10,2018-06-05 13:24:00,2018-06-16 15:20:55,2018-07-18 00:00:00,1,d70f38e7f79c630f8ea00c993897042c,...,576ea0cab426cd8a00fad9a9c90a4494,41213,salvador,BA,41213,-12.929673,-38.445114,salvador,BA,1086.809665
4,acce194856392f074dbf9dada14d8d82,7e20bf5ca92da68200643bda76c504c6,delivered,2018-06-04 00:00:13,2018-06-05 00:35:10,2018-06-05 13:24:00,2018-06-16 15:20:55,2018-07-18 00:00:00,2,9451e630d725c4bb7a5a206b48b99486,...,576ea0cab426cd8a00fad9a9c90a4494,41213,salvador,BA,41213,-12.929673,-38.445114,salvador,BA,1086.809665
